In [ ]:
from openai import OpenAI
from config import OPENAI_API_KEY
import json

client = OpenAI(api_key=OPENAI_API_KEY)

# --- Define functions the AI can call ---
def create_calendar_event(title, date, time, attendee):
    print("\n[Action] Creating Calendar Event:")
    print(f"Title: {title}")
    print(f"Date : {date}")
    print(f"Time : {time}")
    print(f"Attendee: {attendee}")
    print("-" * 40)

def send_email(to, subject, body):
    print("\n[Action] Sending Email:")
    print(f"To     : {to}")
    print(f"Subject: {subject}")
    print(f"Body   : {body}")
    print("-" * 40)

# --- Define tool schemas ---
tools = [
    {
        "type": "function",
        "name": "create_event",
        "description": "Create a calendar event with title, date, time, and attendee.",
        "parameters": {
            "type": "object",
            "properties": {
                "title": {"type": "string"},
                "date": {"type": "string"},
                "time": {"type": "string"},
                "attendee": {"type": "string"}
            },
            "required": ["title", "date", "time", "attendee"]
        }
    },
    {
        "type": "function",
        "name": "send_email",
        "description": "Send an email with a recipient, subject, and body.",
        "parameters": {
            "type": "object",
            "properties": {
                "to": {"type": "string"},
                "subject": {"type": "string"},
                "body": {"type": "string"}
            },
            "required": ["to", "subject", "body"]
        }
    }
]

# --- Initialize conversation ---
messages = [
    {"role": "system", "content": "You are a helpful assistant that can create calendar events and send emails."}
]

def handle_function_call(function_call):
    """Execute a function returned by the model"""
    name = function_call["name"]
    args = json.loads(function_call["arguments"])
    if name == "create_event":
        create_calendar_event(**args)
    elif name == "send_email":
        send_email(**args)

# --- Chat loop ---
while True:
    user_input = input("\nYou: ")
    if user_input.lower() in ["exit", "quit"]:
        break

    messages.append({"role": "user", "content": user_input})

    # --- Stream response with new Responses API ---
    stream = client.responses.create(
        model="gpt-4.1-mini",
        input=messages,
        tools=tools,
        stream=True
    )

    assistant_message = ""
    full_function_call = ""

    for event in stream:
        # Stream partial text output
        if event.type == "response.output_text.delta":
            print(event.delta, end="", flush=True)
            assistant_message += event.delta

        # Collect function call data
        elif event.type == "response.completed":
            # Check if the first output is a function call
            if event.response.output and event.response.output[0].type in ["function_call", "tool_call"]:
                full_function_call = event.response.output[0].arguments

                # Execute function if model returned one
                if full_function_call:
                    function_call = {
                        "name": event.response.output[0].name,
                        "arguments": full_function_call
                    }
                    handle_function_call(function_call)
                else:
                    # No function call was made
                    print("\n[No function call in this response]")



    print("\n")  # newline after streaming

    # Append assistant message to conversation
    messages.append({"role": "assistant", "content": assistant_message})


In [ ]:
# Hello! How can I assist you today? If you want to create a calendar event or send an email, please provide me with the details.

# Add a team meeting on Friday at 10 AM with Alice

# Output Example: 

# [Action] Creating Calendar Event:
# Title: Team Meeting with Alice
# Date : 2024-04-26
# Time : 10:00 AM
# Attendee: Alice
# ----------------------------------------

# Please send an email to alice@example.com with the subject "Project Update" and body "The project meeting is scheduled for tomorrow at 3 PM."

# Output Example: 

# [Action] Sending Email:
# To     : alice@example.com
# Subject: Project Update
# Body   : The project meeting is scheduled for tomorrow at 3 PM.
# ----------------------------------------